In [31]:
import pdfplumber
import re

# location = r"C:\Users\SamsonC\Documents\Accounting\Accounting_AR\invoice_pdf\8713.pdf"
location = r"C:\Users\SamsonC\Documents\Accounting\Accounting_AR\invoice_pdf\salefish-invoice.pdf"


In [32]:
# open and read line from pdf in text
with pdfplumber.open(location) as pdf:
    text = "\n".join(page.extract_text() for page in pdf.pages if page.extract_text())

    print(text)

print()
print("---=-- line splitted --------")
lines = text.split("\n")  
print(lines)
    


Date: 2020-11-30
Invoice
Client: Branthaven Marz Inc.
720 Oval Court
Invoice No: 4560
Burlington, On
L7L 6A9 Agreement No: A0224
Terms: Net 30
Re: #A0224 - Casa De Torri Qty Rate Price
To bill for Hosting and Upgrades as per section 3 of the contract: 1 400.00 400.00
November, 2020
HST On Sales 13.00% 52.00
Total (CDN) $452.00
HST 811629252
It's been a pleasure working with you!

---=-- line splitted --------
['Date: 2020-11-30', 'Invoice', 'Client: Branthaven Marz Inc.', '720 Oval Court', 'Invoice No: 4560', 'Burlington, On', 'L7L 6A9 Agreement No: A0224', 'Terms: Net 30', 'Re: #A0224 - Casa De Torri Qty Rate Price', 'To bill for Hosting and Upgrades as per section 3 of the contract: 1 400.00 400.00', 'November, 2020', 'HST On Sales 13.00% 52.00', 'Total (CDN) $452.00', 'HST 811629252', "It's been a pleasure working with you!"]


In [33]:
# Extract table rows
table_row_regex = r'(.+):\s+(\d+)\s+([\d,.]+)\s+([\d,.]+)'
matches = re.findall(table_row_regex, text)

# Print table data
for match in matches:
    description, qty, rate, price = match
    print(f"Description: {description.strip()}")
    print(f"Qty: {qty}")
    print(f"Rate: {rate}")
    print(f"Price: ${price}")

# Extract tax information
tax_match = re.search(r'HST On Sales ([\d.]+%) ([\d,.]+)', text)
if tax_match:
    tax_rate = tax_match.group(1)
    tax_amount = tax_match.group(2)
    print(f"Tax Rate: {tax_rate}")
    print(f"Tax Amount: ${tax_amount}")

Description: To bill for Hosting and Upgrades as per section 3 of the contract
Qty: 1
Rate: 400.00
Price: 400.00
Tax Rate: 13.00%
Tax Amount: $52.00


In [34]:
# start extracting each invoice item

# extract Date
date_match = re.search(r'Date:\s*(\d{4}-\d{2}-\d{2})', text)
date = date_match.group(1) if date_match else "Date not found"
print(f"Date: {date}")

# extract Invoice Number
invoice_match = re.search(r'Invoice No:\s*(\d+)', text)
invoice_number = invoice_match.group(1) if invoice_match else "Invoice No not found"
print(f"Invoice No: {invoice_number}")

# extract Client Name
client_match = re.search(r'Client:\s*([A-Za-z\s.,-]+)', text)
client_name = client_match.group(1).strip() if client_match else "Client not found"
print(f"Client: {client_name}")

print()

# extract Client Address
extracted_address = None

for i, line in enumerate(lines):
    if client_name in line:
        if i + 1 < len(lines):  # ensure there's a next line
            next_line = lines[i + 1].strip()
            # address validation pattern (assumes addresses start with a number)
            if re.match(r"^\d{1,5}\s[\w\s]+[,\s]*$", next_line):
                extracted_address = next_line
                break

print(f"Address: {extracted_address if extracted_address else 'No address found'}")

unit = re.search(r"^\s*Unit\s*\d+", text, re.MULTILINE)           # Strictly match unit lines
if unit:
    unit_line = unit.group(0).strip()
    print("Unit:", unit_line)

city_test = re.search(r"(?m)^\s*[A-Za-z\s]+(?=, ON)", text)
if city_test:
    print("City:", city_test.group(0))
else:
    print("City not matched.")

province_test = re.search(r"[A-Z]{2}", text)
if province_test:
    print("Province:", province_test.group(0))
else:
    print("Province not matched.")

postal_test = re.search(r"[A-Z]\d[A-Z]\s*\d[A-Z]\d", text)
if postal_test:
    print("Postal Code:", postal_test.group(0))
else:
    print("Postal code not matched.")

print()

# extract header with Agreement No and Client
header_match = re.search(r'Re: #([A-Z0-9]+) - ([A-Za-z\s]+)\s*(Qty\s+Rate\s+Price)', text)
if header_match:
    agreement_number = header_match.group(1).strip()
    client_name = header_match.group(2).strip()
    column_labels = header_match.group(3).strip()
    print(f"Agreement No: {agreement_number}")
    print(f"Client: {client_name}")
    # print(f"Column Labels: {column_labels}")
    print()

else:
    print("Header not found.")
# Extract Table Rows with Multi-line Descriptions
table_data = []
i = 0

while i < len(lines):
    match = re.search(r'(.+?):\s+(\d+)\s+([\d,.]+)\s+([\d,.]+)', lines[i])
    if match:
        description = match.group(1).strip()
        qty = match.group(2)
        rate = match.group(3)
        price = match.group(4)

        # Check if the next line is part of the description (i.e., no quantity/rate/price format)
        next_line_index = i + 1
        while next_line_index < len(lines):
            next_line = lines[next_line_index].strip()
            if not re.search(r'^\d+\s+[\d,.]+\s+[\d,.]+$', next_line):  # If next line is not a row
                description += f" {next_line}"
                next_line_index += 1
            else:
                break

        table_data.append((description, qty, rate, price))
        i = next_line_index  # Move index forward to skip added description lines
    else:
        i += 1  # Continue searching

Date: 2020-11-30
Invoice No: 4560
Client: Branthaven Marz Inc.

Address: 720 Oval Court
City not matched.
Province: HS
Postal Code: L7L 6A9

Agreement No: A0224
Client: Casa De Torri



In [35]:

# Extract only the relevant table section (from "Terms: Net 30" to "HST On Sales")
start_idx = next((i for i, line in enumerate(lines) if "Terms: Net 30" in line), None)
end_idx = next((i for i, line in enumerate(lines) if "HST On Sales" in line), None)

if start_idx is not None and end_idx is not None:
    table_rows = lines[start_idx:end_idx]  # Extract relevant section
else:
    table_rows = []

# Process Table Data
table_data = []
i = 0

while i < len(table_rows):
    row = table_rows[i].strip()

    # Match rows where numbers (Qty, Rate, Price) appear at the end
    match = re.search(r'(.+?)\s+(\d*\.?\d+)\s+([\d,.]+)\s+([\d,.]+)-?', row)
    if match:
        description = match.group(1).strip()
        qty = match.group(2)
        rate = match.group(3)
        price = match.group(4)

        # Check if the next line continues the description (not a new row)
        next_line_index = i + 1
        while next_line_index < len(table_rows):
            next_line = table_rows[next_line_index].strip()
            if not re.search(r'^\d*\.?\d+\s+[\d,.]+\s+[\d,.]+-?$', next_line):  # If next line is NOT a new row
                description += f" {next_line}"
                next_line_index += 1
            else:
                break

        table_data.append((description, qty, rate, price))
        i = next_line_index  # Skip processed lines
    else:
        i += 1  # Continue searching

# Extract HST Row Separately
hst_match = re.search(r'HST On Sales\s+([\d.]+%)\s+([\d,.]+)', text)
if hst_match:
    hst_rate = hst_match.group(1)
    hst_price = hst_match.group(2)
    table_data.append(("HST On Sales", "-", hst_rate, hst_price))  # No qty

# Print Extracted Data
if table_data:
    print("\nExtracted Table Data:")
    for description, qty, rate, price in table_data:
        print(f"Description: {description.strip()}")
        print(f"Qty: {qty}")
        print(f"Rate: {rate}")
        print(f"Price: {price}")
else:
    print("No table data found.")


Extracted Table Data:
Description: To bill for Hosting and Upgrades as per section 3 of the contract: November, 2020
Qty: 1
Rate: 400.00
Price: 400.00
Description: HST On Sales
Qty: -
Rate: 13.00%
Price: 52.00
